In [1]:
from config.Experiment_Config import ExperimentConfig
from exp.trainer import Trainer
import matplotlib.pyplot as plt

/Users/yunkaigao/opt/anaconda3/envs/multi_turn/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
config = ExperimentConfig.get_default_config()

Found existing checkpoint at epoch 2/100. Training will continue from the latest checkpoint.


In [3]:
trainer = Trainer(
    config=config
)

# Train model
print("Starting training...")
metrics_history = trainer.train()

Token indices sequence length is longer than the specified maximum sequence length for this model (1024 > 512). Running this sequence through the model will result in indexing errors


Starting training...
Loading checkpoint: saved_models/BiLSTM_BERT_base_uncased_l2/checkpoints/BiLSTM_BERT_base_uncased_l2_epoch_2.pt
Continuing training from epoch 3
Experiment name: BiLSTM_BERT_base_uncased_l2

Epoch 3/100 - BiLSTM_BERT_base_uncased_l2


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
Training:   0%|          | 0/15 [00:00<?, ?it/s]TOKENIZERS_PARALLELISM=(true | false)
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


In [ ]:
%matplotlib inline

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(trainer.train_losses, label='Training Loss')
plt.plot(trainer.val_losses, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Print final test results
print("\nTest Results:")
test_metrics, _, _ = trainer.evaluate(test_loader, 'test')
for metric, value in test_metrics.items():
    print(f"{metric}: {value:.4f}")